# Sample Calculation
This notebook contains a brief outline of some of the calculations supporting the results of [La Plante et al. (2022)](https://ui.adsabs.harvard.edu/abs/2022arXiv220509770L/abstract). We provide a brief description of the main points of the calculation, and refer the interested reader to the full text for more details.

## `compute_snr.f90`
Most of the calculation is done in `compute_snr.f90`. This file reads in power spectrum files generated from simulations (the 21cm auto spectrum $P_{21}(k_\perp, k_\parallel)$, the galaxy auto spectrum $P_\mathrm{gal}(k_\perp, k_\parallel)$, and the 21cm-galaxy cross spectrum $P_\mathrm{21,gal}(k_\perp, k_\parallel)$), and computes the per-mode signal to noise ratio (S/N). We also read in a few other files that contain information necessary for the calculation. More details can be found in the [README file for the data directory](data/README.md):
- `beam_area.hdf5`: the $\Omega_p$ and $\Omega_{pp}$ quantities for the HERA primary beam. This enters into the calculation of the 21cm noise in Equation (11).
- `nbl_of_u.txt`: the number of baselines in HERA as a function of $u$. This enters into the calculation of the 21cm noise in Equation (11).
- `dndz_ares.txt`: the derivative of the comoving number density of galaxies with respect to redshift, as predicted by <span style="font-variant:small-caps;">ares</span>. This enters into the calculation of the galaxy shot-noise in Equation (14).

A brief outline of the calculation steps are as follows:
- perform necessary cosmology calculations (e.g., comoving distance)
- read in power spectra and auxiliary files
- compute the per-mode S/N as a function of $k_\perp$ and $k_\parallel$
- write out $\mathrm{S/N}(k_\perp, k_\parallel)$

Note that this calculation does not (directly) reference the proposed survey geometry, or account for the number of times a particular mode is sampled by a real-world measurement. It does, however, compute the projected S/N assuming different values for $f_\mathrm{LAE}$ (the effective fraction of overall galaxies that are measured as Ly$\alpha$ emitters in a future Roman survey) and $\sigma_z$ (then uncertainty with which galaxy redshift values are determined). Both of these factors affect the uncertainty of the measured galaxy spectrum $\sigma_\mathrm{gal}$.

## `cumulative_snr.py`
Once we have the per-mode S/N ratio for different observational assumptions, we are able to compute the cumulative S/N for the whole experiment. This is done with `cumulative_snr.py`. We reproduce some of the calculations here, and and show some of the values from Table 1.

### Dependencies:
- `astropy`
- `h5py`
- `matplotlib`
- `numba`
- `numpy`
- `pandas`
- `scipy`
### Optional dependencies:
- `pytables` (for saving table output in HDF5 format)

In [1]:
%matplotlib inline
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numba import njit
from astropy import units, constants
from astropy.cosmology import Planck18
from scipy.interpolate import RectBivariateSpline

In [2]:
# define constants
f21_cgs = 1420405751.768  # 21cm radiation frequency, in Hz
bw_MHz = 6  # observing bandwidth in MHz
dlnkperp = 1e-1  # size of logarithmic k_perp bins
dlnkpara = 1e-1  # size of logarithmic k_para bins
k_max = 1  # maximum k mode, in h/Mpc
hera_FoV = 10  # HERA field of view, in degrees

In [3]:
# define survey parameters
hls_dsq = 2200  # deg^2 footprint for Roman HLS
hera_dsq = 1000  # deg^2 footprint for HERA
cross_dsq = 500  # deg^2 footprint for overlap

spectrum_type = "cross"  # one of: "cross", "21cm", "gal"

In [4]:
def wedge_slope(z):
    """
    Compute the slope of the 21cm wedge as a function of redshift.
    
    This function assumes a "maximal" contamination of the foreground
    wedge, where the foregrounds reach the horizon in delay space. Using
    the slope m, we assume that all modes where k_parallel <= m * k_perp
    are contaminated.
    
    Parameters
    ----------
    z : float
        The redshift of interest.
        
    Returns
    -------
    float
        The slope of the foreground contamination in k_perp/k_parallel
        space.
    """
    dcomoving = Planck18.comoving_distance(z)
    return float(
        dcomoving * Planck18.H(z) / (constants.c * (1 + z))
    )

In [5]:
def lmax_of_z(z, hera_FoV=10):
    """
    Compute the maximum comoving distance for transverse k-modes at redshift z.

    This assumes the field of view of HERA is defined in degrees. The result is
    in comoving Mpc/h and assumes a Planck18 cosmology.

    Parameters
    ----------
    z : float
        Redshift of interest.
    hera_FoV : float, optional
        The field-of-view of the primary beam of HERA, in degrees.

    Returns
    -------
    float:
        The transverse scale corresponding to HERA's FoV in comoving Mpc/h.
    """
    a = 1.0 / (1 + z)
    dA = Planck18.angular_diameter_distance(z).to("Mpc").value
    theta_max = hera_FoV * np.pi / 180.0
    return theta_max * dA / a * Planck18.h

In [6]:
@njit
def calc_nmodes(v_survey, k_perp, k_para, dlnkperp, dlnkpara):
    """
    Compute the number of modes measured in a given Fourier bin.

    Parameters
    ----------
    v_survey : float
        The volume of the survey. Units of comoving length cubed.
    k_perp : float
        The k_perpendicular coordinate. Units of inverse comoving length.
    k_para : float
        The k_parallel coordinate. Units of inverse comoving length.
    dlnkperp : float
        The logarithmic width of the bin in the k_perp direction. Dimensionless.
    dlnkpara : float
        The logarithmic width of the bin in the k_parallel direction.
        Dimensionless.

    Returns
    -------
    float
        The number of Fourier modes that sample this cylindrical bin given the
        proposed survey volume.
    """
    return v_survey / (2 * np.pi)**2 * k_perp**2 * k_para * dlnkperp * dlnkpara

In [7]:
@njit(boundscheck=True)
def nmodes_loop(kperp_vals, kpara_vals, v_survey, nmodes):
    """
    Compute the number of modes for a given Fourier bin using numba.
    
    Parameters
    ----------
    kperp_vals : 1d-array of float
        The list of k_perpendicular coordinates, with shape (n_kperp,). Units
        of inverse comoving length.
    kpara_vals : 1d-array of float
        The list of k_parallel coordinates, with shape (n_kpara,). Units of
        inverse comoving length.
    v_survey : float
        The volume of the survey. Units of comoving length cubed.
    nmodes : 2d-array of float
        The number of modes that sample this cylindrical bin given the proposed
        survey volume. Has shape (n_kperp, n_kpara). Dimensionless.
    """
    for i in range(kperp_vals.shape[0]):
        kperp = kperp_vals[i]
        for j in range(kpara_vals.shape[0]):
            kpara = kpara_vals[j]

            nmodes[i, j] = calc_nmodes(v_survey, kperp, kpara, dlnkperp, dlnkpara)

    return

In [8]:
@njit(boundscheck=True)
def sn_loop(
    kperp_vals, kpara_vals, spn, spectrum_type, slp, kmin_hera
):
    """
    Compute the cumulative S/N ratio given a list of Fourier modes.
    
    Parameters
    ----------
    kperp_vals : 1d-array of float
        The list of k_perpendicular coordinates, with shape (n_kperp,). Units
        of inverse comoving length.
    kpara_vals : 1d-array of float
        The list of k_parallel coordinates, with shape (n_kpara,). Units of
        inverse comoving length.
    spn : 2d-array of float
        The S/N per-mode of a given Fourier mode.
    spectrum_type : str
        The type of spectrum in the `spn` array. Should be one of: "gal", "21cm",
        "cross". If "21cm" or "cross", Fourier modes that are contaminated by the
        foregrounds are excluded from the calculation.
    slp : float
        The slope of the foreground wedge.
    kmin_hera : float
        The minimum k_perpendicular Fourier mode probed by HERA.
        
    Returns
    -------
    cs : float
        The cumulative S/N summed across all modes.
    """
    cs = 0.0
    nkperp = kperp_vals.shape[0]
    nkpara = kpara_vals.shape[0]
    for i in range(nkperp):
        kperp = kperp_vals[i]
        for j in range(nkpara):
            kpara = kpara_vals[j]

            if spectrum_type != "gal":
                # test for wedge cut
                if kpara < kperp * slp:
                    continue
                if kperp < kmin_hera:
                    continue
            cs += spn[i, j]**2

    return cs

In [9]:
# set up calculation
if spectrum_type == "gal":
    area_dsq = hls_dsq
elif spectrum_type == "21cm":
    area_dsq = hera_dsq
elif spectrum_type == "cross":
    area_dsq = cross_dsq
else:
    raise ValueError("spectrum_type must be one of: gal, 21cm, cross")
    
# We assume a square observing area for the purposes of mode counting.
# For galaxies, we use the full footprint. For 21cm + cross spectrum,
# we count the number of non-overlapping HERA fields-of-view that fit
# in the total survey area.
if spectrum_type == "gal":
    theta = np.sqrt(area_dsq) * np.pi / 180.0
else:
    theta = min(hera_FoV, np.sqrt(area_dsq)) * np.pi / 180.0
n_patch = max(int(area_dsq / (hera_FoV**2)), 1)
print("theta: ", theta)
print("n_patch: ", n_patch)

theta:  0.17453292519943295
n_patch:  5


In [10]:
# initialize
table_rows = []

fid_dir = "data/"
dir_list = [fid_dir]
dir_labels = ["Fiducial"]

warm_igm_fn = "snr.hdf5"
fn_list = [warm_igm_fn]
fn_labels = ["saturated"]

wedge_cases = ["horizon", 1.0, 0.5]
wedge_labels = ["horizon", "1.0", "0.5"]

# Loop over histories, observing properties, and spin temperature saturation.
# Note that we only have the fiducial history + saturated IGM in repo
for direc, dir_label in zip(dir_list, dir_labels):
    for fn, fn_label in zip(fn_list, fn_labels):
        # read in data
        snr_fn = os.path.join(direc, fn)
        print(f"reading {snr_fn}...")
        with h5py.File(snr_fn, "r") as h5f:
            zmid_vals = h5f["data/zmid_vals"][()]
            kperp_vals = h5f["data/kperp_vals"][()]
            kpara_vals = h5f["data/kpara_vals"][()]
            snr_array = h5f["data/snr"][()]
            sigma_vals = h5f["header/sigma_z"][()]
            lae_frac = h5f["header/lae_frac"][()]

        # compute the cumulative S/N ratio
        nsigma = sigma_vals.size
        nlae = lae_frac.size

        for isig in range(nsigma):
            sig_val = sigma_vals[isig]
            for ilae in range(nlae):
                f_lae = lae_frac[ilae]
                for slope_val, wedge_label in zip(wedge_cases, wedge_labels):
                    # initialize S/N
                    cs = 0.0
                    for iz, zval in enumerate(zmid_vals):
                        # get observational factors
                        aval = 1.0 / (1 + zval)
                        Lperp_max = (
                            theta
                            * Planck18.angular_diameter_distance(zval).to("Mpc").value
                            / aval
                            * Planck18.h
                        )
                        kperp_min = 2 * np.pi / Lperp_max
                        
                        f0 = f21_cgs * aval
                        f1 = f0 - bw_MHz * 1e6 / 2
                        f2 = f0 + bw_MHz * 1e6 / 2
                        a1 = f1 / f21_cgs
                        a2 = f2 / f21_cgs
                        z1 = 1 / a1 - 1
                        z2 = 1 / a2 - 1
                        d1 = Planck18.comoving_distance(z1).to("Mpc").value
                        d2 = Planck18.comoving_distance(z2).to("Mpc").value
                        Lpara_max = np.abs(d2 - d1) * Planck18.h
                        kpara_min = 2 * np.pi / Lpara_max

                        v_survey = Lperp_max**2 * Lpara_max

                        # ignore redshifts that do not have grism coverage
                        if zval < 7.2:
                            continue
                        if slope_val == "horizon":
                            slp = wedge_slope(zval)
                        else:
                            slp = slope_val
                        hera_lmax = lmax_of_z(zval)
                        kmin_hera = 2 * np.pi / hera_lmax

                        # Build splines
                        # To compute the S/N value at k values that correspond to
                        # our survey modes (rather than our simulated modes), we
                        # compute the S/N ratio, take the log, and then linearly
                        # interpolate in (k_perp, k_parallel) space. We then
                        # exponentiate back to get the S/N value
                        nxvals = np.count_nonzero(kperp_vals[iz, :])
                        nyvals = np.count_nonzero(kpara_vals[iz, :])
                        # See data/README.md for an explanation on snr datasets
                        if spectrum_type == "cross":
                            signal_array = snr_array[iz, ilae, isig, :nxvals, :nyvals, 1]
                            var_array = snr_array[iz, ilae, isig, :nxvals, :nyvals, 2]
                        elif spectrum_type == "21cm":
                            signal_array = snr_array[iz, ilae, isig, :nxvals, :nyvals, 3]
                            var_array = snr_array[iz, ilae, isig, :nxvals, :nyvals, 4]
                        else:
                            signal_array = snr_array[iz, ilae, isig, :nxvals, :nyvals, 5]
                            var_array = snr_array[iz, ilae, isig, :nxvals, :nyvals, 6]

                        if np.any(np.isinf(var_array)):
                            # replace infinite values
                            varmax = np.amax(
                                var_array, initial=0.0, where=np.isfinite(var_array)
                            )
                            var_array = np.where(
                                np.isfinite(var_array), var_array, varmax
                            )
                        spn_spline = RectBivariateSpline(
                            kperp_vals[iz, :nxvals],
                            kpara_vals[iz, :nyvals],
                            np.log(np.abs(signal_array / np.sqrt(var_array))),
                        )

                        # define kbins given survey area
                        nkperp = int(np.log(k_max / kperp_min) / dlnkperp)
                        nkpara = int(np.log(k_max / kpara_min) / dlnkpara)

                        kperp = np.asarray([
                            np.exp((i + 0.5) * dlnkperp) * kperp_min
                            for i in range(nkperp)
                        ])
                        kpara = np.asarray([
                            np.exp((i + 0.5) * dlnkpara) * kpara_min
                            for i in range(nkpara)
                        ])

                        # do array broadcasting
                        spn = np.exp(spn_spline(kperp, kpara, grid=True))
                        nmodes = np.zeros(
                            (kperp.shape[0], kpara.shape[0]), dtype=np.float64
                        )
                        nmodes_loop(kperp, kpara, v_survey, nmodes)
                        spn *= np.sqrt(nmodes)
                        if spectrum_type != "gal":
                            # for 21cm + cross spectra, assume incoherent averaging
                            # across discrete patches
                            spn *= np.sqrt(n_patch)

                        loop_output = sn_loop(
                            kperp, kpara, spn, spectrum_type, slp, kmin_hera
                        )

                        cs += loop_output

                    # add to running list of table rows
                    snr = np.sqrt(cs)
                    data = [dir_label, fn_label, f_lae, sig_val, wedge_label, snr]
                    table_rows.append(data)

reading data/snr.hdf5...


In [11]:
# make a data frame
columns = ["History", "IGM", "f_LAE", "sigma_z", "wedge", "S/N"]
df = pd.DataFrame(data=table_rows, columns=columns)

save_output = True
try:
    import tables
except ImportError:
    print("Install pytables to save table to disk")
    save_output = False
    
if save_output:
    # save it out
    output_file = f"data/cum_snr_nmodes_{spectrum_type}.hdf5"
    print(f"saving {output_file}...")
    df.to_hdf(output_file, key="snr", mode="w")

# print to screen
rows = [
    ["Fiducial", "saturated", 0.1, 0.01, "horizon"],
    ["Fiducial", "saturated", 0.1, 0.01, "1.0"],
    ["Fiducial", "saturated", 0.1, 0.01, "0.5"],
    ["Fiducial", "saturated", 0.1, 0.001, "horizon"],
    ["Fiducial", "saturated", 0.1, 0.1, "horizon"],
    ["Fiducial", "saturated", 1, 0.01, "horizon"],
    ["Fiducial", "saturated", 0.01, 0.01, "horizon"],
]
indices = []
for row in rows:
    selection = df[
        (df["History"] == row[0])
        & (df["IGM"] == row[1])
        & (df["f_LAE"] == row[2])
        & (df["sigma_z"] == row[3])
        & (df["wedge"] == row[4])
    ]
    indices.append(selection.index[0])
# select all indices
print(df.iloc[indices])

saving data/cum_snr_nmodes_cross.hdf5...
     History        IGM  f_LAE  sigma_z    wedge        S/N
12  Fiducial  saturated   0.10    0.010  horizon  11.644026
13  Fiducial  saturated   0.10    0.010      1.0  27.780892
14  Fiducial  saturated   0.10    0.010      0.5  32.420057
3   Fiducial  saturated   0.10    0.001  horizon  14.302854
21  Fiducial  saturated   0.10    0.100  horizon   0.553372
15  Fiducial  saturated   1.00    0.010  horizon  34.503363
9   Fiducial  saturated   0.01    0.010  horizon   3.713312
